In [1]:
# import sys
# import os
# sys.dont_write_bytecode = True
# os.environ["NUMBA_DISABLE_JIT"] = "1"

In [2]:
# import os
# os.environ["NUMBA_DEBUG_CACHE"] = "1"

In [3]:
import numpy as np
import pandas as pd
from my_stuff import MufexKeys
from nb_quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from nb_quantfreedom.nb_enums import DynamicOrderSettingsArrays
from nb_quantfreedom.utils import pretty_qf
from requests import get
from numba import njit, types, typed
from numba.experimental import jitclass
import logging
from nb_quantfreedom.nb_enums import *
from nb_quantfreedom.nb_helper_funcs import dos_cart_product
from nb_quantfreedom.strategies.nb_strategy import (
    strat_bt_create_ind,
    strat_evaluate,
    strat_get_current_ind_settings,
    strat_get_ind_set_str,
    strat_get_total_ind_settings,
)
from nb_quantfreedom.nb_simulate import sim_df_backtest
from nb_quantfreedom.sim_order_records import sim_get_or


mufex_main = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
    use_test_net=False,
)

%load_ext autoreload
%autoreload 2

In [4]:
candles_np = mufex_main.get_candles_df(symbol="BTCUSDT", timeframe="5m", candles_to_dl=2000)

It took 00 mins and 01 seconds to download 2000 candles


In [5]:
mufex_main.set_exchange_settings(
    symbol="BTCUSDT",
    position_mode=PositionModeType.HedgeMode,
    leverage_mode=LeverageModeType.Isolated,
)
backtest_settings = BacktestSettings(
    divide_records_array_size_by=15,
    gains_pct_filter=-90,
    total_trade_filter=60,
    upside_filter=-np.inf,
)
dos_arrays = DynamicOrderSettingsArrays(
    entry_size_asset=np.array([0]),
    max_equity_risk_pct=np.array([6, 9, 12]),
    max_trades=np.array([5]),
    num_candles=np.array([100]),
    risk_account_pct_size=np.array([3]),
    risk_reward=np.array([1, 3, 6]),
    sl_based_on_add_pct=np.array([0.01, 0.03, .06]),
    sl_based_on_lookback=np.array([10, 30]),
    sl_bcb_type=np.array([CandleBodyType.Low]),
    sl_to_be_cb_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct=np.array([0]),
    sl_to_be_ze_type=np.array([0]),
    static_leverage=np.array([0]),
    trail_sl_bcb_type=np.array([CandleBodyType.Low]),
    trail_sl_by_pct=np.array([.1, 0.5, 1.0]),
    trail_sl_when_pct=np.array([1, 2, 4]),
)
static_os = StaticOrderSettings(
    increase_position_type=IncreasePositionType.RiskPctAccountEntrySize,
    leverage_strategy_type=LeverageStrategyType.Dynamic,
    long_or_short=LongOrShortType.Long,
    logger_bool=True,
    pg_min_max_sl_bcb=PriceGetterType.Min,
    sl_to_be_bool=False,
    z_or_e_type=ZeroOrEntryType.Nothing,
    sl_strategy_type=StopLossStrategyType.SLBasedOnCandleBody,
    tp_strategy_type=TakeProfitStrategyType.RiskReward,
    tp_fee_type=TakeProfitFeeType.Limit,
    trail_sl_bool=True,
)
dos_cart_arrays = dos_cart_product(
    dos_arrays=dos_arrays,
)

In [6]:
sim_df_backtest(
    backtest_settings=backtest_settings,
    candles=candles_np,
    dos_cart_arrays=dos_cart_arrays,
    evaluate=strat_evaluate,
    exchange_settings=mufex_main.exchange_settings,
    get_current_ind_settings=strat_get_current_ind_settings,
    get_ind_set_str=strat_get_ind_set_str,
    get_total_ind_settings=strat_get_total_ind_settings,
    ind_creator=strat_bt_create_ind,
    logger_type=LoggerType.Pass,
    starting_equity=1000.0,
    static_os=static_os,
)

Starting the backtest now ... and also here are some stats for your backtest.

Total indicator settings to test: 6
Total order settings to test: 486
Total combinations of settings to test: 2,916
Total candles: 2,000
Total candles to test: 5,832,000


,ind_set_idx,dos_index,total_trades,gains_pct,win_rate,to_the_upside,fees_paid,total_pnl,ending_eq
0,0,54,89.0,-86.20,19.10,0.0,711.941,-862.0195,137.9805
1,0,57,89.0,-89.97,16.85,0.0,704.196,-899.6947,100.3053
2,0,60,64.0,-84.38,17.19,0.0,521.164,-843.8249,156.1751
3,0,72,64.0,-61.28,25.00,0.0,725.261,-612.7868,387.2132
4,0,75,74.0,-80.58,21.62,0.0,717.938,-805.7778,194.2222
...,...,...,...,...,...,...,...,...,...
117,3,450,83.0,-59.16,24.10,0.0,2211.157,-591.5577,408.4423
118,3,453,74.0,-31.82,24.32,0.0,2545.333,-318.1976,681.8024
119,3,454,62.0,-79.98,14.52,0.0,980.466,-799.8293,200.1707
120,3,468,70.0,-49.44,28.57,0.0,2184.759,-494.3766,505.6234


In [7]:
order_records_df = sim_get_or(
    backtest_settings=backtest_settings,
    candles=candles_np,
    dos_cart_arrays=dos_cart_arrays,
    evaluate=strat_evaluate,
    exchange_settings=mufex_main.exchange_settings,
    get_current_ind_settings=strat_get_current_ind_settings,
    get_ind_set_str=strat_get_ind_set_str,
    get_total_ind_settings=strat_get_total_ind_settings,
    ind_creator=strat_bt_create_ind,
    logger_type=LoggerType.Pass,
    starting_equity=1000.0,
    static_os=static_os,
    ind_set_index=0,
    dos_index=0,
)

In [8]:
order_records_df.T

,0,1,2,3,4,5,6,7,8,9,...,240,241,242,243,244,245,246,247,248,249
ind_set_idx,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
or_set_idx,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
bar_idx,100,101,132,135,153,154,154,155,156,164,...,1818,1826,1827,1835,1836,1841,1842,1852,1853,1864
timestamp,1697683500000,1697683800000,1697693100000,1697694000000,1697699400000,1697699700000,1697699700000,1697700000000,1697700300000,1697702700000,...,1698198900000,1698201300000,1698201600000,1698204000000,1698204300000,1698205800000,1698206100000,1698209100000,1698209400000,1698212700000
datetime,2023-10-19 02:45:00,2023-10-19 02:50:00,2023-10-19 05:25:00,2023-10-19 05:40:00,2023-10-19 07:10:00,2023-10-19 07:15:00,2023-10-19 07:15:00,2023-10-19 07:20:00,2023-10-19 07:25:00,2023-10-19 08:05:00,...,2023-10-25 01:55:00,2023-10-25 02:35:00,2023-10-25 02:40:00,2023-10-25 03:20:00,2023-10-25 03:25:00,2023-10-25 03:50:00,2023-10-25 03:55:00,2023-10-25 04:45:00,2023-10-25 04:50:00,2023-10-25 05:45:00
order_status,EntryFilled,EntryFilled,TakeProfitFilled,EntryFilled,StopLossFilled,EntryFilled,StopLossFilled,EntryFilled,EntryFilled,TakeProfitFilled,...,EntryFilled,StopLossFilled,EntryFilled,StopLossFilled,EntryFilled,StopLossFilled,EntryFilled,TakeProfitFilled,EntryFilled,StopLossFilled
equity,1000.0,1000.0,950.3143,950.3143,921.3469,921.3469,893.3533,893.3533,893.3533,854.1981,...,21.0665,20.0602,20.0602,19.0595,19.0595,18.0773,18.0773,17.3029,17.3029,16.1923
available_balance,900.323,799.992,950.3143,862.987,921.3469,783.743,893.3533,751.944,620.453,854.1981,...,18.971,20.0602,16.865,19.0595,14.671,18.0773,14.07,17.3029,15.312,16.1923
cash_borrowed,8530.91,17023.412,NaN,7139.288,NaN,11786.501,NaN,12112.462,23233.898,NaN,...,133.813,NaN,268.805,NaN,375.864,NaN,343.237,NaN,120.97,NaN
cash_used,99.677,200.008,NaN,87.327,NaN,137.604,NaN,141.41,272.901,NaN,...,2.095,NaN,3.196,NaN,4.388,NaN,4.007,NaN,1.99,NaN
